In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from pyDOE import lhs
from Utilities.plotting import newfig, savefig
from mpl_toolkits.mplot3d import Axes3D
import time
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn

In [ ]:
seed = 1234 
random.seed(seed)          # Python標準乱数
np.random.seed(seed)       # NumPy
torch.manual_seed(seed)    # PyTorch (CPU)

In [ ]:
class PhysicsInformedNN:
    # Initialize the class
    def __init__(self, x0, u0, v0, tb, X_f, layers, lb, ub):
        super().__init__()
        #初期条件の構成
        X0 = np.concatenate((x0, 0*x0), 1) # (x0, 0)
        X_lb = np.concatenate((0*tb + lb[0], tb), 1) # (lb[0], tb)
        X_ub = np.concatenate((0*tb + ub[0], tb), 1) # (ub[0], tb)
        
        self.lb = lb
        self.ub = ub

        #時間と空間に分解   
        self.x0 = X0[:,0:1]
        self.t0 = X0[:,1:2]

        self.x_lb = X_lb[:,0:1]#
        self.t_lb = X_lb[:,1:2]

        self.x_ub = X_ub[:,0:1]
        self.t_ub = X_ub[:,1:2]
        
        self.x_f = X_f[:,0:1]
        self.t_f = X_f[:,1:2]
        
        self.u0 = u0
        self.v0 = v0
        
        # Initialize NNs、ニューラルネットワークを構成するパラメータ
        self.layers = layers
        self.weights, self.biases = self.initialize_NN(layers)
        
        # tf Placeholders、出力などを入力するスロット
        """
        self.x0_tf = tf.placeholder(tf.float32, shape=[None, self.x0.shape[1]])
        self.t0_tf = tf.placeholder(tf.float32, shape=[None, self.t0.shape[1]])
        
        self.u0_tf = tf.placeholder(tf.float32, shape=[None, self.u0.shape[1]])
        self.v0_tf = tf.placeholder(tf.float32, shape=[None, self.v0.shape[1]])
        
        self.x_lb_tf = tf.placeholder(tf.float32, shape=[None, self.x_lb.shape[1]])
        self.t_lb_tf = tf.placeholder(tf.float32, shape=[None, self.t_lb.shape[1]])
        
        self.x_ub_tf = tf.placeholder(tf.float32, shape=[None, self.x_ub.shape[1]])
        self.t_ub_tf = tf.placeholder(tf.float32, shape=[None, self.t_ub.shape[1]])
        
        self.x_f_tf = tf.placeholder(tf.float32, shape=[None, self.x_f.shape[1]])
        self.t_f_tf = tf.placeholder(tf.float32, shape=[None, self.t_f.shape[1]])
        """

        self.x0_tor = torch.tensor(self.x0, dtype=torch.float32)
        self.t0_tor = torch.tensor(self.t0, dtype=torch.float32)

        self.u0_tor = torch.tensor(self.u0, dtype=torch.float32)
        self.v0_tor = torch.tensor(self.v0, dtype=torch.float32)

        self.x_lb_tor = torch.tensor(self.x_lb, dtype=torch.float32)
        self.t_lb_tor = torch.tensor(self.t_lb, dtype=torch.float32)

        self.x_ub_tor = torch.tensor(self.x_ub, dtype=torch.float32)
        self.t_ub_tor = torch.tensor(self.t_ub, dtype=torch.float32)
        
        self.x_f_tor = torch.tensor(self.x_f, dtype=torch.float32)
        self.t_f_tor = torch.tensor(self.t_f, dtype=torch.float32)



        
        # Optimizers、L-BFGS-B 最適化
        optimizer_adam = torch.optim.Adam(self.parameters(), lr=1e-3)

        for i in range(10000):
            optimizer_adam.zero_grad()
            self.loss.backward()
            optimizer_adam.step()

        optimizer_lbfgs = torch.optim.LBFGS(
            self.parameters(),
            max_iter=50000,
            max_eval=50000,
            history_size=50,
            tolerance_grad=1e-16,
            tolerance_change=1e-16,
            line_search_fn="strong_wolfe"
        )
            
            


        optimizer_lbfgs.step(self.closure)

        # tf session
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)

              
    def initialize_NN(self, layers):
        weights = nn.ParameterList()
        biases = nn.ParameterList()
        num_layers = len(layers)

        for l in range(num_layers - 1):
            W = nn.Parameter(
                torch.empty(layers[l], layers[l+1])
            )
            nn.init.xavier_normal_(W)  # Xavier初期化

            b = nn.Parameter(
                torch.zeros(1, layers[l+1])
            )

            weights.append(W)
            biases.append(b)

        return weights, biases
        
    
    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1

        # [-1, 1] 正規化
        H = 2.0 * (X - self.lb) / (self.ub - self.lb) - 1.0

        for l in range(num_layers - 2):
            H = torch.tanh(H @ weights[l] + biases[l])

        Y = H @ weights[-1] + biases[-1]
        return Y


    def net_uv(self, x, t):
        x.requires_grad_(True)
        t.requires_grad_(True)

        X = torch.cat([x, t], dim=1)
        uv = self.neural_net(X, self.weights, self.biases)

        u = uv[:, 0:1]
        v = uv[:, 1:2]

        u_x = torch.autograd.grad(
            u, x,
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]

        v_x = torch.autograd.grad(
            v, x,
            grad_outputs=torch.ones_like(v),
            retain_graph=True,
            create_graph=True
        )[0]
        return u, v, u_x, v_x
    
    def net_f_uv(self, x, t):
        # x, t は requires_grad=True である必要がある
        x.requires_grad_(True)
        t.requires_grad_(True)

        # NN を通す
        u, v, u_x, v_x = self.net_uv(x, t)

        # 時間微分
        u_t = torch.autograd.grad(
            u, t,
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]

        v_t = torch.autograd.grad(
            v, t,
            grad_outputs=torch.ones_like(v),
            retain_graph=True,
            create_graph=True
        )[0]

        # 空間2階微分
        u_xx = torch.autograd.grad(
            u_x, x,
            grad_outputs=torch.ones_like(u_x),
            retain_graph=True,
            create_graph=True
        )[0]

        v_xx = torch.autograd.grad(
            v_x, x,
            grad_outputs=torch.ones_like(v_x),
            retain_graph=True,
            create_graph=True
        )[0]

        # PDE residual
        f_u = u_t + 0.5 * v_xx + (u**2 + v**2) * v
        f_v = v_t - 0.5 * u_xx - (u**2 + v**2) * u

        return f_u, f_v


    def callback(self, loss):#繰り返しごとにLOSSを表示
        print('Loss:', loss)
        

        
    def loss_fn(self):
        self.u0_pred, self.v0_pred, _, _ = self.net_uv(self.x0_tor, self.t0_tor)
        self.u_lb_pred, self.v_lb_pred, self.u_x_lb_pred, self.v_x_lb_pred = self.net_uv(self.x_lb_tor, self.t_lb_tor)
        self.u_ub_pred, self.v_ub_pred, self.u_x_ub_pred, self.v_x_ub_pred = self.net_uv(self.x_ub_tor, self.t_ub_tor)
        self.f_u_pred, self.f_v_pred = self.net_f_uv(self.x_f_tor, self.t_f_tor)

        loss = (
            torch.mean((self.u0_tor - self.u0_pred) ** 2) +
            torch.mean((self.v0_tor - self.v0_pred) ** 2) +
            torch.mean((self.u_lb_pred - self.u_ub_pred) ** 2) +
            torch.mean((self.v_lb_pred - self.v_ub_pred) ** 2) +
            torch.mean((self.u_x_lb_pred - self.u_x_ub_pred) ** 2) +
            torch.mean((self.v_x_lb_pred - self.v_x_ub_pred) ** 2) +
            torch.mean(self.f_u_pred ** 2) +
            torch.mean(self.f_v_pred ** 2)
        )
        return loss

    def closure(self):
        self.optimizer_lbfgs.zero_grad()
        loss = self.loss_fn()
        loss.backward()
        return loss


    def train(self, nIter):#Adamで回す繰り返し回数
        
        tf_dict = {self.x0_tf: self.x0, self.t0_tf: self.t0,
                   self.u0_tf: self.u0, self.v0_tf: self.v0,
                   self.x_lb_tf: self.x_lb, self.t_lb_tf: self.t_lb,
                   self.x_ub_tf: self.x_ub, self.t_ub_tf: self.t_ub,
                   self.x_f_tf: self.x_f, self.t_f_tf: self.t_f}#作った入力型にデータを入力
        
        start_time = time.time()
        for it in range(nIter):#Adamで最適化
            self.sess.run(self.train_op_Adam, tf_dict)
            
            # Print、10繰り返しごとに進捗を表示
            if it % 10 == 0:
                elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                print('It: %d, Loss: %.3e, Time: %.2f' % 
                      (it, loss_value, elapsed))
                start_time = time.time()
                                                                                                                          
        self.optimizer.minimize(self.sess, 
                                feed_dict = tf_dict,         
                                fetches = [self.loss], 
                                loss_callback = self.callback)#L-BFGS-B による最適化       
                                    
    
    def predict(self, X_star):#(x, t)
        
        tf_dict = {self.x0_tf: X_star[:,0:1], self.t0_tf: X_star[:,1:2]}#解 u, v の予測
        
        u_star = self.sess.run(self.u0_pred, tf_dict)#u+vi  
        v_star = self.sess.run(self.v0_pred, tf_dict)  
        
        
        tf_dict = {self.x_f_tf: X_star[:,0:1], self.t_f_tf: X_star[:,1:2]}#PDE残差 f_u, f_v の評価
        
        f_u_star = self.sess.run(self.f_u_pred, tf_dict)
        f_v_star = self.sess.run(self.f_v_pred, tf_dict)
               
        return u_star, v_star, f_u_star, f_v_star#予測解とPDE残差

In [ ]:
import torch
import torch.nn as nn
import numpy as np

class PhysicsInformedNN(nn.Module):

    def __init__(self, x0, u0, v0, tb, X_f, layers, lb, ub):
        super().__init__()

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)

        self.lb = torch.tensor(lb, dtype=torch.float32).to(self.device)
        self.ub = torch.tensor(ub, dtype=torch.float32).to(self.device)

        # training data
        self.x0 = torch.tensor(x0[:,0:1], dtype=torch.float32).to(self.device)
        self.t0 = torch.zeros_like(self.x0).to(self.device)

        self.u0 = torch.tensor(u0, dtype=torch.float32).to(self.device)
        self.v0 = torch.tensor(v0, dtype=torch.float32).to(self.device)

        self.x_lb = torch.tensor(0*tb + lb[0], dtype=torch.float32).to(self.device)
        self.t_lb = torch.tensor(tb, dtype=torch.float32).to(self.device)

        self.x_ub = torch.tensor(0*tb + ub[0], dtype=torch.float32).to(self.device)
        self.t_ub = torch.tensor(tb, dtype=torch.float32).to(self.device)

        self.x_f = torch.tensor(X_f[:,0:1], dtype=torch.float32).to(self.device)
        self.t_f = torch.tensor(X_f[:,1:2], dtype=torch.float32).to(self.device)
        

        # NN
        self.weights, self.biases = self.initialize_NN(layers)
        self.to(self.device)

    # ---------------- NN ----------------
    def initialize_NN(self, layers):
        weights = nn.ParameterList()
        biases = nn.ParameterList()

        for l in range(len(layers)-1):
            W = nn.Parameter(torch.empty(layers[l], layers[l+1]))
            nn.init.xavier_normal_(W)
            b = nn.Parameter(torch.zeros(1, layers[l+1]))
            weights.append(W)
            biases.append(b)

        return weights, biases

    def neural_net(self, X):
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for l in range(len(self.weights)-1):
            H = torch.tanh(H @ self.weights[l] + self.biases[l])
        return H @ self.weights[-1] + self.biases[-1]

    def net_uv(self, x, t):
        x.requires_grad_(True)
        t.requires_grad_(True)
        uv = self.neural_net(torch.cat([x, t], dim=1))
        u, v = uv[:,0:1], uv[:,1:2]

        u_x = torch.autograd.grad(u, x, torch.ones_like(u),
                                  create_graph=True)[0]
        v_x = torch.autograd.grad(v, x, torch.ones_like(v),
                                  create_graph=True)[0]
        return u, v, u_x, v_x

    def net_f_uv(self, x, t):
        u, v, u_x, v_x = self.net_uv(x, t)

        u_t = torch.autograd.grad(u, t, torch.ones_like(u),
                                  create_graph=True)[0]
        v_t = torch.autograd.grad(v, t, torch.ones_like(v),
                                  create_graph=True)[0]

        u_xx = torch.autograd.grad(u_x, x, torch.ones_like(u_x),
                                   create_graph=True)[0]
        v_xx = torch.autograd.grad(v_x, x, torch.ones_like(v_x),
                                   create_graph=True)[0]

        f_u = u_t + 0.5*v_xx + (u**2 + v**2)*v
        f_v = v_t - 0.5*u_xx - (u**2 + v**2)*u
        return f_u, f_v

    # ---------------- Loss ----------------
    def loss_fn(self):
        u0, v0, _, _ = self.net_uv(self.x0, self.t0)
        u_lb, v_lb, u_x_lb, v_x_lb = self.net_uv(self.x_lb, self.t_lb)
        u_ub, v_ub, u_x_ub, v_x_ub = self.net_uv(self.x_ub, self.t_ub)
        f_u, f_v = self.net_f_uv(self.x_f, self.t_f)

        loss = torch.mean((u0 - self.u0)**2) +\
            torch.mean((v0 - self.v0)**2) +\
            torch.mean((u_lb - u_ub)**2) +\
            torch.mean((v_lb - v_ub)**2) +\
            torch.mean((u_x_lb - u_x_ub)**2) +\
            torch.mean((v_x_lb - v_x_ub)**2) +\
            torch.mean(f_u**2) +\
            torch.mean(f_v**2)
        

        return (
            torch.mean((u0 - self.u0)**2) +
            torch.mean((v0 - self.v0)**2) +
            torch.mean((u_lb - u_ub)**2) +
            torch.mean((v_lb - v_ub)**2) +
            torch.mean((u_x_lb - u_x_ub)**2) +
            torch.mean((v_x_lb - v_x_ub)**2) +
            torch.mean(f_u**2) +
            torch.mean(f_v**2)
        )

    # ---------------- Training ----------------
    def train_adam(self, n_iter=10000):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        for i in range(n_iter):
            optimizer.zero_grad()
            loss = self.loss_fn()
            loss.backward()
            optimizer.step()
            if i % 100 == 0:
                print(f"Adam {i}, Loss {loss.item():.3e}")

    def train_lbfgs(self):
        optimizer = torch.optim.LBFGS(self.parameters(), max_iter=50000)

        def closure():
            optimizer.zero_grad()
            loss = self.loss_fn()
            loss.backward()
            return loss

        optimizer.step(closure)

    def predict(self, X_star):
        X_star = torch.tensor(X_star, dtype=torch.float32).to(self.device)

        x = X_star[:, 0:1]
        t = X_star[:, 1:2]

        u, v, _, _ = self.net_uv(x, t)
        f_u, f_v = self.net_f_uv(x, t)

        return (
            u.detach().cpu().numpy(),
            v.detach().cpu().numpy(),
            f_u.detach().cpu().numpy(),
            f_v.detach().cpu().numpy()
        )

        
    def train_model(self, nIter):
        self.train()
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)

        for it in range(nIter):
            optimizer.zero_grad()
            loss = self.loss_fn()
            loss.backward()
            optimizer.step()

            if it % 100 == 0:
                print(it, loss.item())


In [ ]:
noise = 0.0        
    
    # Doman bounds
lb = np.array([-5.0, 0.0])
ub = np.array([5.0, np.pi/2])
print(lb)

N0 = 50
N_b = 50
N_f = 20000
layers = [2, 100, 100, 100, 100, 2]
    
data = scipy.io.loadmat('/workspaces/template_pytorch/PINNs-master/main/Data/NLS.mat')

t = data['tt'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = data['uu']
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)
Exact_h = np.sqrt(Exact_u**2 + Exact_v**2)
print(Exact_u.shape)

X, T = np.meshgrid(x,t)
print(X)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact_u.T.flatten()[:,None]
v_star = Exact_v.T.flatten()[:,None]
h_star = Exact_h.T.flatten()[:,None]

###########################

idx_x = np.random.choice(x.shape[0], N0, replace=False)
print(x[0])
x0 = x[idx_x,:]
u0 = Exact_u[idx_x,0:1]
v0 = Exact_v[idx_x,0:1]

idx_t = np.random.choice(t.shape[0], N_b, replace=False)
tb = t[idx_t,:]

X_f = lb + (ub-lb)*lhs(2, N_f)
        
model = PhysicsInformedNN(x0, u0, v0, tb, X_f, layers, lb, ub)

start_time = time.time()
model.train_model(50000)
elapsed = time.time() - start_time
print(f"Training time: {elapsed:.4f}")

model.eval()
u_pred, v_pred, f_u_pred, f_v_pred = model.predict(X_star)

h_pred = np.sqrt(u_pred**2 + v_pred**2)
error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
error_v = np.linalg.norm(v_star-v_pred,2)/np.linalg.norm(v_star,2)
error_h = np.linalg.norm(h_star-h_pred,2)/np.linalg.norm(h_star,2)
print('Error u: %e' % (error_u))
print('Error v: %e' % (error_v))
print('Error h: %e' % (error_h))


U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')
V_pred = griddata(X_star, v_pred.flatten(), (X, T), method='cubic')
H_pred = griddata(X_star, h_pred.flatten(), (X, T), method='cubic')

FU_pred = griddata(X_star, f_u_pred.flatten(), (X, T), method='cubic')
FV_pred = griddata(X_star, f_v_pred.flatten(), (X, T), method='cubic')     



######################################################################
############################# Plotting ###############################
######################################################################    

X0 = np.concatenate((x0, 0*x0), 1) # (x0, 0)
X_lb = np.concatenate((0*tb + lb[0], tb), 1) # (lb[0], tb)
X_ub = np.concatenate((0*tb + ub[0], tb), 1) # (ub[0], tb)
X_u_train = np.vstack([X0, X_lb, X_ub])

fig, ax = newfig(1.0, 0.9)
ax.axis('off')

####### Row 0: h(t,x) ##################    
gs0 = gridspec.GridSpec(1, 2)
gs0.update(top=1-0.06, bottom=1-1/3, left=0.15, right=0.85, wspace=0)
ax = plt.subplot(gs0[:, :])

h = ax.imshow(H_pred.T, interpolation='nearest', cmap='YlGnBu', 
                extent=[lb[1], ub[1], lb[0], ub[0]], 
                origin='lower', aspect='auto')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)

ax.plot(X_u_train[:,1], X_u_train[:,0], 'kx', label = 'Data (%d points)' % (X_u_train.shape[0]), markersize = 4, clip_on = False)

line = np.linspace(x.min(), x.max(), 2)[:,None]
ax.plot(t[75]*np.ones((2,1)), line, 'k--', linewidth = 1)
ax.plot(t[100]*np.ones((2,1)), line, 'k--', linewidth = 1)
ax.plot(t[125]*np.ones((2,1)), line, 'k--', linewidth = 1)    

ax.set_xlabel('$t$')
ax.set_ylabel('$x$')
leg = ax.legend(frameon=False, loc = 'best')
#    plt.setp(leg.get_texts(), color='w')
ax.set_title('$|h(t,x)|$', fontsize = 10)

####### Row 1: h(t,x) slices ##################    
gs1 = gridspec.GridSpec(1, 3)
gs1.update(top=1-1/3, bottom=0, left=0.1, right=0.9, wspace=0.5)

ax = plt.subplot(gs1[0, 0])
ax.plot(x,Exact_h[:,75], 'b-', linewidth = 2, label = 'Exact')       
ax.plot(x,H_pred[75,:], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$|h(t,x)|$')    
ax.set_title('$t = %.2f$' % (t[75]), fontsize = 10)
ax.axis('square')
ax.set_xlim([-5.1,5.1])
ax.set_ylim([-0.1,5.1])

ax = plt.subplot(gs1[0, 1])
ax.plot(x,Exact_h[:,100], 'b-', linewidth = 2, label = 'Exact')       
ax.plot(x,H_pred[100,:], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$|h(t,x)|$')
ax.axis('square')
ax.set_xlim([-5.1,5.1])
ax.set_ylim([-0.1,5.1])
ax.set_title('$t = %.2f$' % (t[100]), fontsize = 10)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.8), ncol=5, frameon=False)

ax = plt.subplot(gs1[0, 2])
ax.plot(x,Exact_h[:,125], 'b-', linewidth = 2, label = 'Exact')       
ax.plot(x,H_pred[125,:], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$|h(t,x)|$')
ax.axis('square')
ax.set_xlim([-5.1,5.1])
ax.set_ylim([-0.1,5.1])    
ax.set_title('$t = %.2f$' % (t[125]), fontsize = 10)